# *Notebook Name*

## Libraries and Configurations

Import configuration files

In [27]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

['../config.ini']

Import **data libraries**

In [28]:
import pandas as pd

Import **other libraries**

In [29]:
from rich.progress import Progress
from rich import traceback

import os

traceback.install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7a495cc6eb10>>

Custom helper scripts

In [30]:
%cd ..
from scripts import plotHelper, encodingHelper
%cd data_exploration_cleaning

/home/bacci/COMPACT/notebooks
/home/bacci/COMPACT/notebooks/data_exploration_cleaning


## Import Data

Importing every `CSV` file in the `binary` folder, merge them and create the labelled dataframe.

In [31]:
# Base directory containing the folders
base_dir = config["DEFAULT"]["binary_path"]

# Initialize an empty dictionary to store DataFrames
dataframes = {}

# Traverse the directory structure
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".csv"):
            # Construct the full file path
            file_path = os.path.join(root, file)

            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Store the DataFrame in the dictionary with a unique key (e.g., file name)
            dataframes[file] = df

In [32]:
bin_df = pd.concat(dataframes.values(), ignore_index=True)

Export to CSV in `interim` folder

In [33]:
bin_df.to_csv("../../data/interim/binary_df_raw.csv", index=False)

Set every column's type to string in order to avoid conversions and misrepresentation of the binary data.

In [34]:
bin_df = bin_df.astype(str)

In [35]:
bin_df

,mac,e_id_ssid,len_ssid,ssid,e_id_sup_rates,len_sup_rates,supported_rates,e_id_ext_sup_rates,len_ext_sup_rates,ext_sup_rates,...,len_vht_cap,vht_cap,e_id_vst,len_vst,vst,e_id_ext_tags,len_ext_tags,ext_tags,frame_check_seq,label
0,1a:e6:5a:fe:34:4c,0,0,nan,1,100,00000010000001000000101100010110,110010.0,1000.0,0000110000010010000110000010010000110000010010...,...,0000000000000000000010001000010000000000000000...,nan,nan,nan,nan,nan,nan,nan,00101110110101000110100111001111,iPhone7_F
1,1a:e6:5a:fe:34:4c,0,0,nan,1,100,00000010000001000000101100010110,110010.0,1000.0,0000110000010010000110000010010000110000010010...,...,0000000000000000000010001000010000000000000000...,nan,nan,nan,nan,nan,nan,nan,11010111100011001000011011110001,iPhone7_F
2,1a:e6:5a:fe:34:4c,0,0,nan,1,100,00000010000001000000101100010110,110010.0,1000.0,0000110000010010000110000010010000110000010010...,...,0000000000000000000010001000010000000000000000...,nan,nan,nan,nan,nan,nan,nan,11010111100011001000011011110001,iPhone7_F
3,1a:e6:5a:fe:34:4c,0,0,nan,1,100,00000010000001000000101100010110,110010.0,1000.0,0000110000010010000110000010010000110000010010...,...,0000000000000000000010001000010000000000000000...,nan,nan,nan,nan,nan,nan,nan,11000100001011000110000000100001,iPhone7_F
4,1a:e6:5a:fe:34:4c,0,0,nan,1,100,00000010000001000000101100010110,110010.0,1000.0,0000110000010010000110000010010000110000010010...,...,0000000000000000000010001000010000000000000000...,nan,nan,nan,nan,nan,nan,nan,11000100001011000110000000100001,iPhone7_F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76699,da:a1:19:45:40:f0,0,0,nan,1,100,00000010000001000000101100010110,110010.0,1000.0,0000110000010010000110000010010000110000010010...,...,0000000000000000000010100000001000000001,nan,11011101.0,111.0,0000000001010000111100100000100000000000011000...,nan,nan,nan,10110101111111110110100110111000,XiaomiRedmi4_B
76700,da:a1:19:45:40:f0,0,110,001100010011000100110001001101110111001101111000,1,100,00000010000001000000101100010110,110010.0,1000.0,0000110000010010000110000010010000110000010010...,...,0000000000000000000010100000001000000001,nan,11011101.0,111.0,0000000001010000111100100000100000000000011000...,nan,nan,nan,11111010001100111100010011110000,XiaomiRedmi4_B
76701,da:a1:19:45:40:f0,0,1001,0010000101101111011100000011000001110011011100...,1,100,00000010000001000000101100010110,110010.0,1000.0,0000110000010010000110000010010000110000010010...,...,0000000000000000000010100000001000000001,nan,11011101.0,111.0,0000000001010000111100100000100000000000011000...,nan,nan,nan,11101110101101011101100100100001,XiaomiRedmi4_B
76702,da:a1:19:45:40:f0,0,1000,0101011001101111011001000110000101100110011011...,1,100,00000010000001000000101100010110,110010.0,1000.0,0000110000010010000110000010010000110000010010...,...,0000000000000000000010100000001000000001,nan,11011101.0,111.0,0000000001010000111100100000100000000000011000...,nan,nan,nan,01011111010111110111011001010011,XiaomiRedmi4_B
